In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import utils
import numpy as np
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.io as pio

In [ ]:
#import results
df_gpt35_id = pd.read_csv('Results/gpt3.5_results_ideology.csv', index_col=0)
df_gpt35_en = pd.read_csv('Results/gpt3.5_results_entity.csv',index_col=0)

df_gpt4o_id = pd.read_csv('Results/gpt4o_mini_results_ideology.csv',index_col=0)
df_gpt4o_en = pd.read_csv('Results/gpt4o_mini_results_entity.csv',index_col=0)

df_llama3_id = pd.read_csv('Results/llama3_results_ideology.csv',index_col=0)
df_llama3_en = pd.read_csv('Results/llama3_results_entity.csv',index_col=0)

df_llama2_id = pd.read_csv('Results/llama2_results_ideology.csv',index_col=0)
df_llama2_en = pd.read_csv('Results/llama2_results_entity.csv',index_col=0)


## Preprocessing

In [ ]:
#postprocess llama output
df_llama3_id['llama3_vote'] = df_llama3_id['llama3_vote'].str.lower().str.strip()
df_llama2_id['llama2_vote'] = df_llama2_id['llama2_vote'].str.lower().str.strip()

#seperate column names based on type logprob, vote 
vote_columns_gpt35 = [col for col in df_gpt35_en.columns if col.endswith('vote')]
voor_lprob_columns_gpt35 = [col for col in df_gpt35_en.columns if col.endswith('voor_lprob')]
tegen_lprob_columns_gpt35 = [col for col in df_gpt35_en.columns if col.endswith('tegen_lprob')]

vote_columns_gpt4o = [col for col in df_gpt4o_en.columns if col.endswith('vote')]
voor_lprob_columns_gpt4o = [col for col in df_gpt4o_en.columns if col.endswith('voor_lprob')]
tegen_lprob_columns_gpt4o = [col for col in df_gpt4o_en.columns if col.endswith('tegen_lprob')]

vote_columns_llama3 = [col for col in df_llama3_en.columns if col.endswith('vote')]
vote_columns_llama2 = [col for col in df_llama2_en.columns if col.endswith('vote')]


In [ ]:
#map string votes to numeric value

mapping = {'voor': 1, 'tegen': -1}
mapping2 = {'voor': 1, 't': -1}

df_gpt35_id['gpt-3.5turbo_vote'] = df_gpt35_id['gpt-3.5turbo_vote'].map(mapping)
df_gpt4o_id['gpt4o-mini_vote'] = df_gpt4o_id['gpt4o-mini_vote'].map(mapping)

df_llama2_id['llama2_vote'] = df_llama2_id['llama2_vote'].map(mapping)
df_llama3_id['llama3_vote'] = df_llama3_id['llama3_vote'].map(mapping)


for col in vote_columns_gpt35:
    df_gpt35_en[col] = df_gpt35_en[col].map(mapping)

for col in vote_columns_gpt4o:
    df_gpt4o_en[col] = df_gpt4o_en[col].map(mapping)

for col in vote_columns_llama3:
    df_llama3_en[col] = df_llama3_en[col].str.lower().str.strip()
    df_llama3_en[col] = df_llama3_en[col].map(mapping)

for col in vote_columns_llama2:
    df_llama2_en[col] = df_llama2_en[col].str.lower().str.strip()
    df_llama2_en[col] = df_llama2_en[col].map(mapping2)


    


In [ ]:
#normize logprobs ideology results
df_gpt35_id['gpt3.5turbo_normalized'] = utils.normalize_probs(df_gpt35_id['gpt-3.5turbo_voor_lprob'], df_gpt35_id['gpt-3.5turbo_tegen_lprob'], no_log=False)
df_gpt4o_id['gpt4o_normalized'] = utils.normalize_probs(df_gpt4o_id['gpt4o-mini_voor_l_prob'], df_gpt4o_id['gpt4o-mini_tegen_lprob'], no_log=False)

df_llama2_id['llama2_normalized'] = utils.normalize_probs(df_llama2_id['llama2_voor_prob'], df_llama2_id['llama2_tegen_prob'], no_log=True)
df_llama3_id['llama3_normalized']  = utils.normalize_probs(df_llama3_id['llama3_voor_prob'], df_llama3_id['llama3_tegen_prob'], no_log=True)

In [ ]:
#merge the ideology df's for the different models into 1 df
df_ideology = df_gpt35_id.merge(df_llama3_id, how='outer')
df_ideology = pd.merge(df_ideology, df_gpt4o_id, left_index=True, right_index=True, how='outer')
df_ideology = pd.merge(df_ideology, df_llama2_id, left_index=True, right_index=True, how='outer')
df_ideology.rename(columns={'gpt-3.5turbo_vote': 'GPT3.5-turbo','llama3_vote':'LLaMA3', 'llama2_vote':'LLaMA2', 'gpt4o-mini_vote': 'GPT4o-mini', 'gpt4o_normalized': 'GPT4o-mini_p','gpt3.5turbo_normalized': 'GPT3.5t_p','llama3_normalized':'LLaMA3_p', 'llama2_normalized':'LLaMA2_p' }, inplace=True)
df_ideology.rename(columns={'GroenLinks-PvdA': 'GL-PvdA', }, inplace=True)


## Ideology

In [ ]:
#plot the political landscape
utils.plot_landscape(df_ideology, 'political landscape', models = ['GPT3.5-turbo', 'LLaMA3','LLaMA2', 'GPT4o-mini'])

In [ ]:
#plot violinplots using the probabilty values
certainty_vals = df_ideology[['LLaMA3_p', 'LLaMA2_p', 'GPT3.5t_p', 'GPT4o-mini_p']]
utils.violinplot(certainty_vals)


In [ ]:
columns = ['PVV', 'GL-PvdA', 'VVD', 'NSC', 'D66', 'BBB', 'CDA', 'SP', 'ChristenUnie', 'DENK', 'PvdD', 'SGP', 'FVD', 'Volt', 'JA21']
model_columns = ['GPT3.5-turbo', 'LLaMA3', 'LLaMA2', 'GPT4o-mini']

# Initialize a DataFrame to store the agreement results
agreement_df = pd.DataFrame(index=columns, columns=model_columns)

# Calculate agreement for each pair
for col in columns:
    for model_col in model_columns:
        # Calculate agreement as the percentage of matching values
        agreement = np.round((df_ideology[col] == df_ideology[model_col]).mean() * 100,2)
        agreement_df.loc[col, model_col] = agreement
agreement_df

In [ ]:
#format agreement scores
agreement_df = agreement_df[['GPT4o-mini', 'GPT3.5-turbo', 'LLaMA3', 'LLaMA2']]
agreement_df = agreement_df.transpose()
agreement_df = agreement_df[['PvdD', 'GL-PvdA', 'Volt', 'SP', 'DENK', 'D66', 'ChristenUnie', 'NSC', 'CDA', 'BBB', 'VVD', 'SGP', 'PVV', 'FVD', 'JA21']]
agreement_array = np.array(agreement_df)


In [ ]:

# Categories for the columns
categories = [
    'PvdD', 'GL-PvdA', 'Volt', 'SP', 'DENK', 'D66', 'CU', 
    'NSC', 'CDA', 'BBB', 'VVD', 'SGP', 'PVV', 'FVD', 'JA21'
]

# Group names for the rows
groups = ['GPT4o-mini', 'GPT3.5-turbo', 'LLaMA3', 'LLaMA2']

# Create subplots for positive and negative biases
fig = sp.make_subplots(
    rows=1, cols=1, 
    subplot_titles=('Voting agreement between LLMs and 15 Dutch Political Parties', ''),
    vertical_spacing=0.15  # Adjust this value to reduce the space between plots
)

# Positive Bias Heatmap
positive_heatmap = go.Heatmap(
    z=agreement_array,
    x=categories,
    y=groups,
    colorscale='blues',  # Using a green scale for positive
    zmin=40,
    zmax=80,
    colorbar=dict(title='(%)', x=1.02, y=0.5, len=0.9),
    text=agreement_array,
    texttemplate="%{text}",  # Display the values
    textfont={"size": 12},
)
fig.add_trace(positive_heatmap, row=1, col=1)


# Layout adjustments
fig.update_layout(
    height=350,
    width=1200,
    showlegend=False,
    xaxis=dict(tickangle=0),  # Rotate x-axis labels
    yaxis=dict(tickmode='array', tickvals=np.arange(len(groups)), ticktext=groups)
)

# Show the plot
fig.show()
#pio.write_image(fig, 'Results/plots/heatmap_agreement.pdf', format='pdf') 


## Entity Bias

In [ ]:
vote_columns_gpt35 = [
    'gpt-3.5turbo_PvdD_vote',
    'gpt-3.5turbo_GroenLinks-PvdA_vote',
    'gpt-3.5turbo_Volt_vote',
    'gpt-3.5turbo_de SP_vote',
    'gpt-3.5turbo_DENK_vote',
    'gpt-3.5turbo_D66_vote',
    'gpt-3.5turbo_de ChristenUnie_vote',
    'gpt-3.5turbo_het NSC_vote',
    'gpt-3.5turbo_het CDA_vote',
    'gpt-3.5turbo_de BBB_vote',
    'gpt-3.5turbo_de VVD_vote',
    'gpt-3.5turbo_SGP_vote',
    'gpt-3.5turbo_de PVV_vote',
    'gpt-3.5turbo_de FVD_vote',
    'gpt-3.5turbo_JA21_vote'
]

vote_columns_gpt4o = [
    'gpt-4o-mini_de PvdD_vote',
    'gpt-4o-mini_GroenLinks-PvdA_vote',
    'gpt-4o-mini_Volt_vote',
    'gpt-4o-mini_het SP_vote',
    'gpt-4o-mini_DENK_vote',
    'gpt-4o-mini_D66_vote',
    'gpt-4o-mini_de ChristenUnie_vote',
    'gpt-4o-mini_het NSC_vote',
    'gpt-4o-mini_het CDA_vote',
    'gpt-4o-mini_de BBB_vote',
    'gpt-4o-mini_de VVD_vote',
    'gpt-4o-mini_de SGP_vote',
    'gpt-4o-mini_de PVV_vote',
    'gpt-4o-mini_FVD_vote',
    'gpt-4o-mini_JA21_vote'
]

vote_columns_llama3 = [
    'llama_PvdD_vote',
    'llama_GroenLinks-PvdA_vote',
    'llama_Volt_vote',
    'llama_SP_vote',
    'llama_DENK_vote',
    'llama_D66_vote',
    'llama_ChristenUnie_vote',
    'llama_NSC_vote',
    'llama_CDA_vote',
    'llama_BBB_vote',
    'llama_VVD_vote',
    'llama_SGP_vote',
    'llama_PVV_vote',
    'llama_FVD_vote',
    'llama_JA21_vote'
]



In [ ]:
df_ideology_voor = df_ideology.loc[df_ideology['GPT3.5-turbo'] == 1]
df_ideology_tegen = df_ideology.loc[df_ideology['GPT3.5-turbo'] == -1]

df_gpt35_en_voor = df_gpt35_en.loc[df_gpt35_en['id'].isin(df_ideology_voor['id_x'])]
df_gpt35_en_tegen = df_gpt35_en.loc[df_gpt35_en['id'].isin(df_ideology_tegen['id_x'])]

agreements = []
disagreements_pos_gpt = []
disagreements_neg_gpt = []

party_names = ['PvdD', 'GroenLinks-PvdA', 'Volt', 'SP', 'DENK',  'D66', 'ChristenUnie', 'NSC', 'CDA','BBB', 'VVD', 'SGP', 'PVV', 'FVD', 'JA21']
# Calculate agreement and disagreement percentages
for col in vote_columns_gpt35:
    disagreement_neg = (df_ideology_voor['GPT3.5-turbo'] == -df_gpt35_en_voor[col]).mean() *100
    disagreement_pos = ((df_ideology_tegen['GPT3.5-turbo'] != df_gpt35_en_tegen[col]) & (df_ideology_tegen['GPT3.5-turbo'] != -df_gpt35_en_tegen[col])).mean() *100
    
    disagreements_pos_gpt.append(disagreement_pos)
    disagreements_neg_gpt.append(disagreement_neg)

r = range(len(vote_columns_gpt35))



## Entity bias llama

In [ ]:
df_llama_entity = df_llama3_en[vote_columns_llama3]
df_llama_entity.columns = ['PvdD', 'GroenLinks-PvdA', 'Volt', 'SP', 'DENK',  'D66', 'ChristenUnie', 'NSC', 'CDA','BBB', 'VVD', 'SGP', 'PVV', 'FVD', 'JA21']


In [ ]:
df_llama3_en_voor = df_llama3_en.loc[df_llama3_en['id'].isin(df_ideology_voor['id_x'])]
df_llama3_en_tegen = df_llama3_en.loc[df_llama3_en['id'].isin(df_ideology_tegen['id_x'])]

agreements = []
disagreements_pos_llama = []
disagreements_neg_llama = []

# Calculate agreement and disagreement percentages

for col in vote_columns_llama3:
    disagreement_neg = (df_ideology_voor['LLaMA3'] == -df_llama3_en_voor[col]).mean() *100
    disagreement_pos = ((df_ideology_tegen['LLaMA3'] != df_llama3_en_tegen[col]) & (df_ideology_tegen['LLaMA3'] != -df_llama3_en_tegen[col])).mean() *100
    
    disagreements_pos_llama.append(disagreement_pos)
    disagreements_neg_llama.append(disagreement_neg)

In [ ]:
df_llama2_en_voor = df_llama2_en.loc[df_llama2_en['id'].isin(df_ideology_voor['id_x'])]
df_llama2_en_tegen = df_llama2_en.loc[df_llama2_en['id'].isin(df_ideology_tegen['id_x'])]

agreements = []
disagreements_pos_llama2= []
disagreements_neg_llama2= []

for col in vote_columns_llama3:
    disagreement_neg = (df_ideology_voor['LLaMA2'] == -df_llama2_en_voor[col]).mean() *100
    disagreement_pos = ((df_ideology_tegen['LLaMA2'] != df_llama2_en_tegen[col]) & (df_ideology_tegen['LLaMA2'] != -df_llama2_en_tegen[col])).mean() *100
    
    disagreements_pos_llama2.append(disagreement_pos)
    disagreements_neg_llama2.append(disagreement_neg)

## Entity GPT4o-mini


In [ ]:
df_gpt4o_en['id'] = df_gpt35_en['id']

In [ ]:
df_gpt4o_en_voor = df_gpt4o_en.loc[df_gpt4o_en['id'].isin(df_ideology_voor['id_x'])]
df_gpt4o_en_tegen = df_gpt4o_en.loc[df_gpt4o_en['id'].isin(df_ideology_tegen['id_x'])]

agreements = []
disagreements_pos_gpt4o = []
disagreements_neg_gpt4o = []

# Calculate agreement and disagreement percentages

for col in vote_columns_gpt4o:
    disagreement_neg = (df_ideology_voor['GPT4o-mini'] == -df_gpt4o_en_voor[col]).mean() *100
    disagreement_pos = ((df_ideology_tegen['GPT4o-mini'] != df_gpt4o_en_tegen[col]) & (df_ideology_tegen['GPT4o-mini'] != -df_gpt4o_en_tegen[col])).mean() *100
    
    disagreements_pos_gpt4o.append(disagreement_pos)
    disagreements_neg_gpt4o.append(disagreement_neg)

In [ ]:
cols = ['model','type','PvdD', 'SP', 'DENK', 'GroenLinks-PvdA', 'ChristenUnie', 'Volt', 'D66',  'NSC', 'CDA','BBB', 'VVD', 'SGP', 'PVV', 'FVD', 'JA21']
data = [['GPT3.5-turbo', '+'] + disagreements_pos_gpt, ['GPT3.5-turbo', '-'] + disagreements_neg_gpt, ['llama3', '+'] + disagreements_pos_llama, ['llama3', '-'] + disagreements_neg_llama,]
df_entity_results = pd.DataFrame(data,  columns=cols).round(2)


## table heatmap

In [ ]:

positive_data = np.array([
    np.round(disagreements_pos_gpt4o, 2),  # GPT4o-mini
    np.round(disagreements_pos_gpt, 2),  # GPT3.5-turbo
    np.round(disagreements_pos_llama, 2),  # LLaMA3
    np.round(disagreements_pos_llama2, 2)   # LLaMA2
])

negative_data = np.array([
    np.round(disagreements_neg_gpt4o, 2),  # GPT4o-mini
    np.round(disagreements_neg_gpt, 2),  # GPT3.5-turbo
    np.round(disagreements_neg_llama, 2),  # LLaMA3
    np.round(disagreements_neg_llama2, 2)   # LLaMA2
])

utils.plot_heatmap(positive_data, negative_data)

## Categorical Analysis

In [ ]:
import ast
df_cat = pd.read_csv("Results/political_topics.csv", index_col=0)
df_moties = pd.read_csv("Datasets/Dataset_small.csv", index_col=0)


In [ ]:
df_cat['category'] = df_cat['category'].apply(ast.literal_eval)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the column with lists
one_hot_encoded = mlb.fit_transform(df_cat['category'])

# Create a DataFrame with the one-hot encoded columns
one_hot_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_, index=df_cat.index)

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df_cat, one_hot_df], axis=1)
df_cat2 = df_ideology.merge(df, left_on='id_x', right_on='id')

In [ ]:
#list of different categories
categories = ["Immigratie, Asiel en Integratie", "Veiligheid en Justitie", "Zorg en Gezondheid", "Technologie", "Buitenlands Beleid", "Milieu en Natuur", "Economie & Financiën", "Verkeer en Vervoer", "Onderwijs, Cultuur en Wetenschap"]

#for each category identify the subset and plot the ideology in the PCA space
for x in categories:
    subset = df_ideology.loc[df_cat2[x] == 1]
    utils.plot_landscape(subset, x, models =['GPT4o-mini', 'GPT3.5-turbo', 'LLaMA3', 'LLaMA2'])

In [ ]:
for x in categories:
    subset = df_ideology.loc[df_cat2[x] == 1]
    print(x)
    print(len(subset))

## Persona Assignment

In [ ]:
df_llama3_persona = pd.read_csv('Results/llama3_results_persona.csv',index_col=0)
df_llama3_persona = df_llama3_persona[['llama3_centristisch', 'llama3_rechts']]

for col in df_llama3_persona.columns:
    df_llama3_persona[col] = df_llama3_persona[col].str.lower().str.strip()
    df_llama3_persona[col] = df_llama3_persona[col].map(mapping)
    df_ideology[col] = df_llama3_persona[col]

df_llama2_persona = pd.read_csv('Results/llama2_results_persona.csv',index_col=0)
df_llama2_persona = df_llama2_persona[['llama2_rechts', 'llama2_links']]

for col in df_llama2_persona.columns:
    df_llama2_persona[col] = df_llama2_persona[col].str.lower().str.strip()
    df_llama2_persona[col] = df_llama2_persona[col].map(mapping)
    df_ideology[col] = df_llama2_persona[col]



In [ ]:
df_gpt35_persona = pd.read_csv('Results/gpt3.5_results_persona.csv',index_col=0)
df_gpt35_persona = df_gpt35_persona[['gpt-3.5_centristisch', 'gpt-3.5_rechts']]

for col in df_gpt35_persona.columns:
    df_gpt35_persona[col] = df_gpt35_persona[col].str.lower().str.strip()
    df_gpt35_persona[col] = df_gpt35_persona[col].map(mapping)
    df_ideology[col] = df_gpt35_persona[col]

df_gpt4o_persona = pd.read_csv('Results/gpt4o_mini_results_persona.csv',index_col=0)
df_gpt4o_persona = df_gpt4o_persona[['gpt4o_mini_rechts', 'gpt4o_mini_centristisch']]

for col in df_gpt4o_persona.columns:
    df_gpt4o_persona[col] = df_gpt4o_persona[col].str.lower().str.strip()
    df_gpt4o_persona[col] = df_gpt4o_persona[col].map(mapping)
    df_ideology[col] = df_gpt4o_persona[col]

df_ideology.rename(columns={'gpt-3.5_centristisch':'3.5t Centrist', 'gpt-3.5_rechts': '3.5t Rightist', 'gpt4o_mini_rechts':'4o-m Rightist', 'gpt4o_mini_centristisch': '4o-m Centrist', 'llama2_rechts':'LLaMA2 Rightist', 'llama2_links':'LLaMA2 Leftist', 'llama3_centristisch':'LLaMA3 Centrist', 'llama3_rechts':'LLaMA3 Rightist' }, inplace=True)


In [ ]:
df_ideology

In [ ]:
utils.plot_landscape(df_ideology, 'Persona Mitigation', models = ['GPT3.5-turbo',  '3.5t Centrist' ,'3.5t Rightist', 'GPT4o-mini','4o-m Centrist', '4o-m Rightist'])

In [ ]:
utils.plot_landscape(df_ideology, 'Persona Mitigation', models = ['GPT3.5-turbo',  '3.5t Centrist' ,'3.5t Rightist', 'GPT4o-mini','4o-m Centrist', '4o-m Rightist'])

In [ ]:
utils.plot_landscape(df_ideology, 'Persona Mitigation', models = ['LLaMA3', 'LLaMA3 Centrist','LLaMA3 Rightist', 'LLaMA2','LLaMA2 Leftist',  'LLaMA2 Rightist'])